In [1]:
# Token
INT = 'INT'
STRING = 'STRING'
ID = 'ID'
ASSIGN = 'ASSIGN'
PLUS = 'PLUS'
MINUS = 'MINUS'
MUL = 'MUL'
DIV = 'DIV'
LPAREN = 'LPAREN'
RPAREN = 'RPAREN'
SEMI = 'SEMI'
PRINT = 'PRINT'
SCAN = 'SCAN'
INT_CONST = 'INT_CONST'
STRING_LITERAL = 'STRING_LITERAL'
EOF = 'EOF'

class Lexer:
    def _init_(self, text):
        self.text = text
        self.pos = 0
        self.current_char = self.text[self.pos]

    def advance(self):
        self.pos += 1
        if self.pos < len(self.text):
            self.current_char = self.text[self.pos]
        else:
            self.current_char = None

    def skip_whitespace(self):
        while self.current_char is not None and self.current_char.isspace():
            self.advance()

    def get_string_literal(self):
        result = ''
        self.advance()  
        while self.current_char is not None and self.current_char != '"':
            result += self.current_char
            self.advance()
        self.advance()  
        return result

    def get_next_token(self):
        while self.current_char is not None:
            if self.current_char.isspace():
                self.skip_whitespace()
                continue

            if self.current_char.isdigit():
                value = ''
                while self.current_char is not None and self.current_char.isdigit():
                    value += self.current_char
                    self.advance()
                return (INT_CONST, int(value))

            if self.current_char.isalpha():
                value = ''
                while self.current_char is not None and (self.current_char.isalnum() or self.current_char == '_'):
                    value += self.current_char
                    self.advance()
                if value == 'int':
                    return (INT, value)
                if value == 'string':
                    return (STRING, value)
                if value == 'printf':
                    return (PRINT, value)
                if value == 'scanf':
                    return (SCAN, value)
                return (ID, value)

            if self.current_char == '+':
                self.advance()
                return (PLUS, '+')

            if self.current_char == '-':
                self.advance()
                return (MINUS, '-')

            if self.current_char == '*':
                self.advance()
                return (MUL, '*')

            if self.current_char == '/':
                self.advance()
                return (DIV, '/')

            if self.current_char == '=':
                self.advance()
                return (ASSIGN, '=')

            if self.current_char == '(':
                self.advance()
                return (LPAREN, '(')

            if self.current_char == ')':
                self.advance()
                return (RPAREN, ')')

            if self.current_char == ';':
                self.advance()
                return (SEMI, ';')

            if self.current_char == '"':
                string_value = self.get_string_literal()
                return (STRING_LITERAL, string_value)

            raise Exception(f"Invalid character: {self.current_char}")

        return (EOF, None)
    
    # Parser class
class Parser:
    def _init_(self, lexer):
        self.lexer = lexer
        self.current_token = self.lexer.get_next_token()

    def eat(self, token_type):
        if self.current_token[0] == token_type:
            self.current_token = self.lexer.get_next_token()
        else:
            raise Exception(f"Unexpected token {self.current_token[1]}")

    def factor(self):
        token = self.current_token
        if token[0] == INT_CONST:
            self.eat(INT_CONST)
            return ('int', token[1])
        elif token[0] == STRING_LITERAL:
            self.eat(STRING_LITERAL)
            return ('string', token[1])
        elif token[0] == ID:
            self.eat(ID)
            return ('var', token[1])
        elif token[0] == LPAREN:
            self.eat(LPAREN)
            expr = self.expr()
            self.eat(RPAREN)
            return expr
        else:
            raise Exception(f"Unexpected factor {token[0]}")

    def term(self):
        node = self.factor()
        while self.current_token[0] in (MUL, DIV):
            token = self.current_token
            if token[0] == MUL:
                self.eat(MUL)
            elif token[0] == DIV:
                self.eat(DIV)
            node = (token[0], node, self.factor())
        return node

    def expr(self):
        node = self.term()
        while self.current_token[0] in (PLUS, MINUS):
            token = self.current_token
            if token[0] == PLUS:
                self.eat(PLUS)
            elif token[0] == MINUS:
                self.eat(MINUS)
            node = (token[0], node, self.term())
        return node

    def assignment(self):
        var = self.current_token
        self.eat(ID)
        self.eat(ASSIGN)
        expr = self.expr()
        return ('assign', var[1], expr)

    def declaration(self):
        if self.current_token[0] == INT:
            self.eat(INT)
            var = self.current_token
            self.eat(ID)
            if self.current_token[0] == ASSIGN:
                self.eat(ASSIGN)
                expr = self.expr()
                return ('declare_assign', 'int', var[1], expr)
            return ('declare', 'int', var[1])
        elif self.current_token[0] == STRING:
            self.eat(STRING)
            var = self.current_token
            self.eat(ID)
            if self.current_token[0] == ASSIGN:
                self.eat(ASSIGN)
                expr = self.expr()
                return ('declare_assign', 'string', var[1], expr)
            return ('declare', 'string', var[1])

    def printf(self):
        self.eat(PRINT)
        self.eat(LPAREN)
        expr = self.expr()
        self.eat(RPAREN)
        return ('printf', expr)

    def scanf(self):
        self.eat(SCAN)
        self.eat(LPAREN)
        var = self.current_token
        self.eat(ID)
        self.eat(RPAREN)
        return ('scanf', var[1])

    def statement(self):
        if self.current_token[0] in (INT, STRING):
            return self.declaration()
        elif self.current_token[0] == ID:
            return self.assignment()
        elif self.current_token[0] == PRINT:
            return self.printf()
        elif self.current_token[0] == SCAN:
            return self.scanf()
        else:
            raise Exception(f"Invalid statement {self.current_token[0]}")

    def parse(self):
        statements = []
        while self.current_token[0] != EOF:
            stmt = self.statement()
            self.eat(SEMI)
            statements.append(stmt)
        return statements

class Interpreter:
    def _init_(self, parser):
        self.parser = parser
        self.symbol_table = {}

    def eval_expr(self, node):
        if node[0] == 'int':
            return node[1]
        elif node[0] == 'string':
            return node[1]
        elif node[0] == 'var':
            var_name = node[1]
            if var_name not in self.symbol_table:
                raise Exception(f"Variable '{var_name}' not defined")
            return self.symbol_table[var_name]
        elif node[0] == PLUS:
            left = self.eval_expr(node[1])
            right = self.eval_expr(node[2])
            if isinstance(left, int) and isinstance(right, int):
                return left + right
            elif isinstance(left, str) and isinstance(right, str):
                return left + right
            else:
                raise Exception("Type mismatch in expression")
        elif node[0] == MINUS:
            return self.eval_expr(node[1]) - self.eval_expr(node[2])
        elif node[0] == MUL:
            return self.eval_expr(node[1]) * self.eval_expr(node[2])
        elif node[0] == DIV:
            return self.eval_expr(node[1]) // self.eval_expr(node[2])  

    def exec_stmt(self, stmt):
        if stmt[0] == 'declare':
            self.symbol_table[stmt[2]] = 0 if stmt[1] == 'int' else ''
        elif stmt[0] == 'declare_assign':
            self.symbol_table[stmt[2]] = self.eval_expr(stmt[3])
        elif stmt[0] == 'assign':
            self.symbol_table[stmt[1]] = self.eval_expr(stmt[2])
        elif stmt[0] == 'printf':
            print(self.eval_expr(stmt[1]))
        elif stmt[0] == 'scanf':
            var_name = stmt[1]
            if var_name not in self.symbol_table:
                raise Exception(f"Variable '{var_name}' not defined")
            
            if isinstance(self.symbol_table[var_name], int):
                while True:
                    try:
                        self.symbol_table[var_name] = int(input(f"Enter value for {var_name}: "))
                        break
                    except ValueError:
                        print("Invalid input. Please enter a valid integer.")
            elif isinstance(self.symbol_table[var_name], str):
                self.symbol_table[var_name] = input(f"Enter value for {var_name}: ")

def main():
    user_input_lines = []
    while True:
        line = input()
        if line == "":
            break
        user_input_lines.append(line)
    
    text = '\n'.join(user_input_lines)

    lexer = Lexer(text)
    parser = Parser(lexer)
    statements = parser.parse()
    interpreter = Interpreter(parser)
    for stmt in statements:
        interpreter.exec_stmt(stmt)

if __name__ == "_main_":
    main()